In [7]:
import platform
print (platform.architecture ())

('64bit', 'WindowsPE')


In [2]:
import requests
from copy import copy


def dfs(category_id):
    url = "https://api.itemscout.io/api/category/{0}/subcategories".format(category_id)
    response = requests.get(url = url)
    
    for d in response.json()['data']:

        d_id = d['id']
        d_name = d['name']
        d_is_leaf = d['is_leaf']
        
        if(d_is_leaf):
            leaf_id = copy(d_id)
            leaf_name = copy(d_name)
            leaf_category.append([leaf_id,leaf_name])
            
        else:
            dfs(d_id)
    
paths = []
leaf_category = []
starting_category = [1,2,3,4,5,8,9] #출산/육아(6) , 식품 제외(7)


first_category = [1]

for category in first_category:
    dfs(category)



print(len(leaf_category))
#1 : 패션의류 --> 68개 
#2 : 패션잡화 --> 264개
#3 : 화장품/미용 --> 157개
#4 : 디지털/가전 --> 681개
#5 : 가구/인테리어 --> 354개
#6 : 출산/육아 (X)
#7 : 식품 (X)
#8 : 스포츠/레저 --> 카테고리 : 480개, 키워드 : 154207개 --> 필터 적용 : 20000개
#9 : 생활/건강 --> 카테고리 : 1227개, 키워드 : 465817개 --> 필터 적용 : 61751개

#app 1개당 기본 api 12500개 
#계정 1개당 app 10개, api 125000개

68


In [3]:
import json
import urllib.request
from collections import OrderedDict
from tqdm import notebook
from joblib import Parallel, delayed
from time import sleep
from config import *
from infsell import *



def extract_titles(category,client_auth):
    
    category_id = category[0]
    category_name = category[1]
    
    url_data = "https://api.itemscout.io/api/category/{0}/data".format(category_id)
    url_brands = "https://api.itemscout.io/api/category/{0}/brands".format(category_id)
    while True:	
        try: 
            response_data = requests.post(url_data, json=PAYLOAD)
            break
        except Exception as e:
            print(e)
            pass
    response_brands = requests.get(url_brands)
    data = response_data.json()['data']['data']
    brands = response_brands.json()['data']
    brands = remove_a_word(brands)
    
    keywords = get_keywords_database(data)
    keywords_nb = remove_brands(keywords,brands)
    
    for keyword in keywords_nb[:5]:
        encText = urllib.parse.quote(keyword)
        url_items = "https://openapi.naver.com/v1/search/shop.json?display=80&query=" + encText # JSON 결과
        titles = []
        
        try:
            items = get_items(url_items,client_auth)
            
            for item in items:
                title = item['title'].replace('<b>','').replace('</b>','')
                titles.append(title)
            
            info_keyword={}
            info_keyword['category'] = category_name
            info_keyword['keyword'] = keyword
            info_keyword['titles'] = titles

            titles_data.append(info_keyword)

        except Exception as e:
            print('error ',e, ' 키워드 : ', keyword)
            pass
            
       
        

#멀티코어 개수
num_core = 12

cnt_keyword=0
path_auth='auth_api.txt'
client_auth = get_auth_api(path_auth)
titles_data = [] 

with Parallel(n_jobs=num_core, require='sharedmem') as parallel:
    parallel(delayed(extract_titles)(category,client_auth)for category in notebook.tqdm(leaf_category[:1]))
    

  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
print(len(titles_data))

5


In [5]:
copy = titles_data

print(titles_data, copy)

[{'category': '코디세트', 'keyword': '니트투피스', 'titles': ['여름 니트 투피스 세트 퍼프 소매 라운드 반팔 밴딩 롱 니트 치마 세트 하객 셋업 5COLOR', '여성 여름 비스코스 플리츠 니트 투피스 셋업 상하세트', '여름 반팔 가디건 스커트 여성 니트 투피스 세트', '여름니트원피스 투피스 반팔니트 롱스커트세트 S84765', '여름니트투피스 반팔 니트 스커트 투피스 세트', '어니스트 펀칭 니트 가디건 여름 투피스 SET', '스카시 진주 여름 니트 투피스 세트', '바니 썸머 반팔 니트 가디건 롱스커트 투피스 세트', '여름 언발 루즈핏 반팔 니트 투피스 세트 셋업 봄 여름 3COLOR', '[쇼핑라이브할인] 썸머 니트 투피스 세트 (가성비굿)', '썸머니트 투피스 루즈핏 와이드팬츠 바지세트', '배색 골지니트 가디건 반바지 투피스세트', '여름 투피스 가오리 루즈핏 니트 스커트 투피스 세트 셋업', '[강추]니트투피스세트 반팔가디건세트 미샤set', '노블리카 세트구성 클레오 반팔 니트 플레어 스커트 투피스 상하의세트', '큐피트 배색 니트 투피스', '앙코르 디어 여름 니트 투피스 세트', '디프니 이듀니트투피스SET', '샤틴 펀칭패턴 니트 투피스 S232K958', '데일리 여름 니트 투피스 세트 (핑크, 크림-2 color)', '주아르 SET 브이넥 반팔 보석 가디건 스커트 니트 투피스 세트', '나비올라 명품스타일투피스 아이보리 여름니트투피스', '모네 라인 배색 트임 탑 루즈핏 하이웨스트 밴딩 와이드 여름 니트 투피스 세트', '모니카제이 와플 골지 나시 니트투피스', '알로하 골지 체크 니트 투피스 원피스 가디건세트', '베이시코 크롭 배색 니트 투피스 세트', '와이드 가디건 바지 가을 간절기 세트 니트투피스', '여름 니트 투피스 플리츠 와이드 팬츠세트', '여름 니트 반바지 세트 브이넥 꾸안꾸 니트투피스 셋업', 'ab.f.z 에이비에프지 망사반팔니트투피스세트-반팔니트 APW2QWZ1E', 'SE

In [4]:


while True:
    try :
        with open('database.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
    
    except :
        with open('test3.json', 'w',encoding='utf-8') as f:
            f.write(json.dumps(titles_data, ensure_ascii=False,indent=4))


FileNotFoundError: [Errno 2] No such file or directory: 'database.json'

In [4]:

path_database = "database6.json"

with open(path_database, 'w',encoding='utf-8') as f:
    f.write(json.dumps(titles_data, ensure_ascii=False,indent=4))


MemoryError: 

In [11]:
with open('database5.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


In [13]:
print(len(data))